# OAlley API example 1 - basic requests

This tutorial will show you how to use the [OAlley API](https://api.oalley.fr) to compute and display isochronous aeras.

You will need to install python packages `folium` `requests` and  `polyline`.

```
pip install folium requests polyline
```

Also, you will need an API key. Go to [api.oalley.fr](https://api.oalley.fr) and signup. It's free. Each new user receives a key with 1000 requests available.

Copy your key and paste it in the code below.

In [1]:
from IPython.display import HTML
import folium
import requests
import polyline
from urllib.parse import urlencode

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-API-KEY"

## Build the URL to request one zone

In this example, we prepare a request for computing a 10 min isochronous area around [Orly Airport](http://tools.wmflabs.org/geohack/geohack.php?language=fr&pagename=Orly-Sud_%28Orlyval%29&params=48.729027_N_2.369972_E_type:railwaystation)

In [2]:
point = {
    'lat': 48.729027,
    'lng': 2.369972,
    'duration' : 10 * 60
}
 
url = baseurl + appkey + method + urlencode(point)

## Execute the request

In [3]:
res = requests.get(url)
body = res.json()

if res.status_code != 200:
    print(body)

# Convert the polyline to JSON 
isozone = polyline.decode(body['polyline'])

# Trick to close the polyline, until folium library supports it
isozone.append(isozone[0])

isozone


[(48.72926, 2.4265),
 (48.73417, 2.42201),
 (48.73819, 2.41608),
 (48.7433, 2.41776),
 (48.74989, 2.42032),
 (48.76034, 2.42546),
 (48.76802, 2.42698),
 (48.77364, 2.42397),
 (48.77645, 2.41742),
 (48.78326, 2.41485),
 (48.78346, 2.40591),
 (48.78506, 2.40011),
 (48.78421, 2.39284),
 (48.79343, 2.38917),
 (48.79452, 2.38297),
 (48.79328, 2.37571),
 (48.80279, 2.37041),
 (48.81334, 2.36146),
 (48.82118, 2.35159),
 (48.81344, 2.34392),
 (48.81012, 2.33645),
 (48.80226, 2.33128),
 (48.786, 2.33191),
 (48.78265, 2.32607),
 (48.78203, 2.31713),
 (48.77528, 2.31358),
 (48.77412, 2.30337),
 (48.76825, 2.29074),
 (48.76252, 2.28686),
 (48.74084, 2.32938),
 (48.73809, 2.32563),
 (48.7346, 2.3104),
 (48.72905, 2.326),
 (48.72391, 2.31324),
 (48.71635, 2.31556),
 (48.71299, 2.31574),
 (48.70716, 2.31878),
 (48.70164, 2.31879),
 (48.70146, 2.32883),
 (48.69825, 2.33344),
 (48.69298, 2.33438),
 (48.69429, 2.34207),
 (48.68271, 2.33906),
 (48.67758, 2.34253),
 (48.68036, 2.35017),
 (48.6776, 2.35492

## Display the zone in a map

The `folium` python package provides a straightforward manner to display maps and plot things in it. 

In [5]:
mymap = folium.Map(location=[46, 2], zoom_start=5)

folium.PolyLine(locations=isozone,weight=5).add_to(mymap)

mymap.fit_bounds(mymap.get_bounds())

mymap

## Compute multiple isochronous areas

It is also possible to request multiples zones from the server. The next cell shows a complete example.


In [6]:
durations = [60 * 5, 60 * 10, 60 * 20, 60 * 30, 60 * 50]

point = {
    'lat': 48.729027,
    'lng': 2.369972,
    'duration': 0.0
}
 
# Build urls    
urls = [baseurl + appkey + method + urlencode(dict(point, duration=duration)) for duration in durations]

zones = []

# Execute them one at a time
for url in urls:
    r = requests.get(url)
    body = r.json()
    isozone = polyline.decode(body['polyline'])
    # Trick to close the polyline, until folium library supports it
    isozone.append(isozone[0])
    zones.append(isozone)

# Plot the result
mymap = folium.Map(location=[46, 2], zoom_start=5)
for zone in zones:
    folium.PolyLine(locations=zone,weight=5).add_to(mymap)

mymap.fit_bounds(mymap.get_bounds())
mymap


## Next

So far, we have requested isochronous areas one at a time. 

If you need to compute a lot more isochronous, you might be interested in the [next](https://nbviewer.jupyter.org/github/overdrivr/interactive-geospatial-analysis/blob/master/Oalley_example_async.ipynb) example to speed up the process.

![isocronous areas aliens meme](./isochronous-areas.jpg)